<h1><b>Лабораторная работа №5 ч.2 </b></h1>
Анализ ансамблевых методов машинного обучения

1. Загрузка и подготовка данных

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from gmdh import Combi, Multi, Mia, Ria
import warnings
warnings.filterwarnings('ignore')

In [29]:
# Загрузка данных
data = pd.read_csv('water_pollution_disease.csv')

# Просмотр первых строк данных
print(data.head())


     Country   Region  Year Water Source Type  Contaminant Level (ppm)  \
0     Mexico    North  2015              Lake                     6.06   
1     Brazil     West  2017              Well                     5.24   
2  Indonesia  Central  2022              Pond                     0.24   
3    Nigeria     East  2016              Well                     7.91   
4     Mexico    South  2005              Well                     0.12   

   pH Level  Turbidity (NTU)  Dissolved Oxygen (mg/L)  Nitrate Level (mg/L)  \
0      7.12             3.93                     4.28                  8.28   
1      7.84             4.79                     3.86                 15.74   
2      6.43             0.79                     3.42                 36.67   
3      6.71             1.96                     3.12                 36.92   
4      8.16             4.22                     9.15                 49.35   

   Lead Concentration (µg/L)  ...  Cholera Cases per 100,000 people  \
0        

In [30]:
# Основная информация о данных
print("\nИнформация о данных:")
print(data.info())


Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 24 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Country                                        3000 non-null   object 
 1   Region                                         3000 non-null   object 
 2   Year                                           3000 non-null   int64  
 3   Water Source Type                              3000 non-null   object 
 4   Contaminant Level (ppm)                        3000 non-null   float64
 5   pH Level                                       3000 non-null   float64
 6   Turbidity (NTU)                                3000 non-null   float64
 7   Dissolved Oxygen (mg/L)                        3000 non-null   float64
 8   Nitrate Level (mg/L)                           3000 non-null   float64
 9   Lead Concentration (µg/L)     

In [31]:
# Проверка пропущенных значений
print("\nПропущенные значения:")
print(data.isnull().sum())



Пропущенные значения:
Country                                            0
Region                                             0
Year                                               0
Water Source Type                                  0
Contaminant Level (ppm)                            0
pH Level                                           0
Turbidity (NTU)                                    0
Dissolved Oxygen (mg/L)                            0
Nitrate Level (mg/L)                               0
Lead Concentration (µg/L)                          0
Bacteria Count (CFU/mL)                            0
Water Treatment Method                           747
Access to Clean Water (% of Population)            0
Diarrheal Cases per 100,000 people                 0
Cholera Cases per 100,000 people                   0
Typhoid Cases per 100,000 people                   0
Infant Mortality Rate (per 1,000 live births)      0
GDP per Capita (USD)                               0
Healthcare Access Index

In [32]:
# Описательная статистика
print("\nОписательная статистика:")
print(data.describe())


Описательная статистика:
              Year  Contaminant Level (ppm)     pH Level  Turbidity (NTU)  \
count  3000.000000              3000.000000  3000.000000      3000.000000   
mean   2012.012667                 4.954390     7.255847         2.480023   
std       7.229287                 2.860072     0.720464         1.419984   
min    2000.000000                 0.000000     6.000000         0.000000   
25%    2006.000000                 2.560000     6.630000         1.257500   
50%    2012.000000                 4.950000     7.280000         2.460000   
75%    2018.000000                 7.400000     7.870000         3.660000   
max    2024.000000                10.000000     8.500000         4.990000   

       Dissolved Oxygen (mg/L)  Nitrate Level (mg/L)  \
count              3000.000000            3000.00000   
mean                  6.492850              25.08025   
std                   2.027966              14.50517   
min                   3.000000               0.05000   


2.Работа с данными

In [33]:
# Заполнение пропусков медианными значениями
numeric_cols = data.select_dtypes(include=[np.number]).columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].median())

Кодирование категориальных переменных

In [34]:
cat_cols = ['Country', 'Region', 'Water Source Type', 'Water Treatment Method']
data = pd.get_dummies(data, columns=cat_cols, drop_first=True)

3. Подготовка данных для моделирования

Выделение признаков и целевой переменной (Infant Mortality Rate) --- Коэффициент младенческой смертности


In [35]:
X = data.drop('Infant Mortality Rate (per 1,000 live births)', axis=1)
y = data['Infant Mortality Rate (per 1,000 live births)']

Масштабирование признаков


In [36]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Разделение на обучающую и тестовую выборки

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

4. Построение и оценка моделей

In [38]:
# Определение базовых моделей для стекинга
estimators = [
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
]

# Создание и обучение модели стекинга
stacking = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)
stacking.fit(X_train, y_train)

# Прогнозирование и оценка
y_pred_stack = stacking.predict(X_test)
rmse_stack = np.sqrt(mean_squared_error(y_test, y_pred_stack))
r2_stack = r2_score(y_test, y_pred_stack)

print("\nСтекинг:")
print(f"RMSE: {rmse_stack:.4f}")
print(f"R²: {r2_stack:.4f}")


Стекинг:
RMSE: 28.3694
R²: -0.0072


4.2. Многослойный перцептрон (MLP)

In [39]:
# Создание и обучение MLP
mlp = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42
)
mlp.fit(X_train, y_train)

# Прогнозирование и оценка
y_pred_mlp = mlp.predict(X_test)
rmse_mlp = np.sqrt(mean_squared_error(y_test, y_pred_mlp))
r2_mlp = r2_score(y_test, y_pred_mlp)

print("\nМногослойный перцептрон:")
print(f"RMSE: {rmse_mlp:.4f}")
print(f"R²: {r2_mlp:.4f}")


Многослойный перцептрон:
RMSE: 38.0136
R²: -0.8083


4.3 Методы МГУА (GMDH)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Уменьшение данных до 10% от исходного размера
def reduce_data_size(X, y, fraction=0.1):
    _, X_reduced, _, y_reduced = train_test_split(
        X, y, 
        train_size=fraction, 
        random_state=42, 
        stratify=None
    )
    return X_reduced, y_reduced

# Применение к обучающим данным
X_train_reduced, y_train_reduced = reduce_data_size(X_train, y_train)

# Преобразование данных для GMDH (требует numpy массивы)
X_train_gmdh = np.array(X_train_reduced)
y_train_gmdh = np.array(y_train_reduced)
X_test_gmdh = np.array(X_test)

# Линейный метод COMBI
try:
    combi = Combi()
    combi.fit(X_train_gmdh, y_train_gmdh)
    y_pred_combi = combi.predict(X_test_gmdh)
    rmse_combi = np.sqrt(mean_squared_error(y_test, y_pred_combi))
    r2_combi = r2_score(y_test, y_pred_combi)

    print("\nМГУА (COMBI) на 10% данных:")
    print(f"RMSE: {rmse_combi:.4f}")
    print(f"R²: {r2_combi:.4f}")
except Exception as e:
    print(f"\nОшибка при выполнении COMBI: {str(e)}")
    print("Попробуйте установить библиотеку gmdh или использовать альтернативные методы")

# Нелинейный метод MIA
try:
    mia = Mia()
    mia.fit(X_train_gmdh, y_train_gmdh)
    y_pred_mia = mia.predict(X_test_gmdh)
    rmse_mia = np.sqrt(mean_squared_error(y_test, y_pred_mia))
    r2_mia = r2_score(y_test, y_pred_mia)

    print("\nМГУА (MIA) на 10% данных:")
    print(f"RMSE: {rmse_mia:.4f}")
    print(f"R²: {r2_mia:.4f}")
except Exception as e:
    print(f"\nОшибка при выполнении MIA: {str(e)}")
    print("Попробуйте установить библиотеку gmdh или использовать альтернативные методы")

# Альтернатива если GMDH не работает
if 'rmse_combi' not in locals():
    print("\nИспользуем Ridge и RandomForest как альтернативу GMDH")
    
    from sklearn.linear_model import Ridge
    from sklearn.ensemble import RandomForestRegressor
    
    # Линейная модель (аналог COMBI)
    ridge = Ridge()
    ridge.fit(X_train_reduced, y_train_reduced)
    y_pred_ridge = ridge.predict(X_test)
    rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
    r2_ridge = r2_score(y_test, y_pred_ridge)
    
    print("\nRidge (линейная альтернатива COMBI):")
    print(f"RMSE: {rmse_ridge:.4f}")
    print(f"R²: {r2_ridge:.4f}")
    
    # Нелинейная модель (аналог MIA)
    rf = RandomForestRegressor(n_estimators=50, random_state=42)
    rf.fit(X_train_reduced, y_train_reduced)
    y_pred_rf = rf.predict(X_test)
    rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
    r2_rf = r2_score(y_test, y_pred_rf)
    
    print("\nRandomForest (нелинейная альтернатива MIA):")
    print(f"RMSE: {rmse_rf:.4f}")
    print(f"R²: {r2_rf:.4f}")

f5. Сравнение моделей:

In [ ]:
# Создание таблицы результатов
results = pd.DataFrame({
    'Метод': ['Стекинг', 'Многослойный перцептрон', 'МГУА (COMBI)', 'МГУА (MIA)'],
    'RMSE': [rmse_stack, rmse_mlp, rmse_combi, rmse_mia],
    'R²': [r2_stack, r2_mlp, r2_combi, r2_mia]
})

print("\nСравнение моделей:")
print(results.sort_values('R²', ascending=False))

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.bar(results['Метод'], results['R²'])
plt.title('Сравнение моделей по метрике R²')
plt.ylabel('R²')
plt.xticks(rotation=45)
plt.show()